In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#- Describe the content of the dataset and its goals

#A: The contents of the dataset patient history and vitals of Pima Indians that are possible risk factors in developing
# diabetes.  I suppose the goal is to identify which patterns matter most when predicting diabetes, so that it can
# diagnosed earlier.

#- Describe the features and formulate a hypothesis on which might be relevant in predicting diabetes
#A: See #7 below: (Taken from data description) Taking a look at the standard deviation, I feel like the glucose
# tolerance test and serum insulin test will be the most valuable features given that they having the highest
# variance.  The number of times pregnant seem very high for certain patients (max 17 times).


#7. For Each Attribute: (all numeric-valued)
#   1. Number of times pregnant - How many times the patient has been pregnant.
#   2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test - Your blood sugar level after given
#      a controlled amount of glucose.
#   3. Diastolic blood pressure (mm Hg) - Your resting blood pressure (when your heart is not flexed).
#   4. Triceps skin fold thickness (mm) - The thickness of your underarm skin.
#   5. 2-Hour serum insulin (mu U/ml)
#   6. Body mass index (weight in kg/(height in m)^2)
#   7. Diabetes pedigree function
#   8. Age (years) - Your age.
#   9. Class variable (0 or 1)


#- Describe the missing/NULL values. Decide if you should impute or drop them and justify your choice.
#Looking at the min values, I suppose a patient could have never been pregnant, but the following features should have
# min values > 0.00 : plasma_glucose, diastolic_pressure, triceps_skin_fold, insulin, bmi
# Looking at the 25% triceps_skin_fold and insulin are 0.00, telling me that there is a lot of missing data in those
#features. I would likely drop those.



attributes = ['times_pregnant', 'plasma_glucose', 'diastolic_pressure', 'triceps_skin_fold', 'insulin', 'bmi', 'diabetes_pedigree','age','class']
data = pd.read_csv('pima-indians-diabetes.data.txt', sep=',', names=attributes)
data.describe()
#data.head(768)


,times_pregnant,plasma_glucose,diastolic_pressure,triceps_skin_fold,insulin,bmi,diabetes_pedigree,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [37]:
data.drop('triceps_skin_fold', axis=1, inplace=True)
data.drop('insulin', axis=1, inplace=True)
data.describe()

,times_pregnant,plasma_glucose,diastolic_pressure,bmi,diabetes_pedigree,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,67.100000,2.420000,81.000000,1.000000


In [45]:
#- Come up with a benchmark for the minimum performance that an algorithm should have on this dataset
#Based on the mean, 34.9% of patients have diabetes.

In [65]:
#- What's the best performance you can get with kNN? Is kNN a good choice for this dataset?

data = data.dropna()
y = data['class']
X = data.drop('class', axis=1)
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

knn = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train) 
y_pred = knn.predict(X_test) 
y_train_pred = knn.predict(X_train) 

print metrics.classification_report(y_test, y_pred, target_names=['No Diabetes', 'Diabetes']) 
#precision
#1 0.72
#2 0.73
#3 0.77
#4 0.77
#5 0.77
#6 0.75
#7 0.73
#8 0.71
#9 0.75
#10 0.73

#f1
#1 0.67
#2 0.65
#3 0.73
#4 0.73
#5 0.76
#6 0.76
#7 0.69
#8 0.71
#9 0.75
#10 0.70

#With neighbors=5, we get the highest precision and f1 score.

             precision    recall  f1-score   support

No Diabetes       0.84      0.86      0.85       132
   Diabetes       0.67      0.65      0.66        60

avg / total       0.79      0.79      0.79       192



In [69]:
#- What's the best performance you can get with Naive Bayes? Is NB a good choice for this dataset?
from sklearn.grid_search import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import ShuffleSplit

cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0)
parameters = {'alpha': np.linspace(1e-4, 1, 10)}
nbayes = MultinomialNB()
clf = GridSearchCV(nbayes, parameters, cv=cv)
clf.fit(X_train, y_train)

GridSearchCV(cv=ShuffleSplit(576, n_iter=10, test_size=0.2, random_state=0),
       error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e-04,   1.11200e-01,   2.22300e-01,   3.33400e-01,
         4.44500e-01,   5.55600e-01,   6.66700e-01,   7.77800e-01,
         8.88900e-01,   1.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [70]:
#- What's the best performance you can get with Logistic Regression? Is LR a good choice for this dataset?
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
# transform our training features
X_train_std = stdsc.fit_transform(X_train)
# transform the testing features in the same way
X_test_std = stdsc.transform(X_test)

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(penalty='l2', C=10)
logreg.fit(X_train_std, y_train)
y_pred = logreg.predict(X_test_std)
print metrics.classification_report(y_test, y_pred, target_names=['No Diabetes', 'Diabetes']) 

#With a precision of 0.78 and and an f1 of 0.77, this is slightly better than kNN.

             precision    recall  f1-score   support

No Diabetes       0.86      0.93      0.89       132
   Diabetes       0.82      0.67      0.73        60

avg / total       0.85      0.85      0.84       192



In [58]:
#- What's the best performance you can get with Random Forest? Is RF a good choice for this dataset?
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

model = RandomForestClassifier(n_estimators = 20)
model.fit(X, y)

scores = cross_val_score(model, X, y, scoring='accuracy')
print('CV accuracy {}, Average accuracy {}'.format(scores, scores.mean()))

for n_trees in range(1, 100, 10):
    model = RandomForestClassifier(n_estimators = n_trees)
    scores = cross_val_score(model, X, y, scoring='accuracy')
    print('Estimators {}, CV accuracy {}, Average accuracy {}'.format(n_trees, scores, scores.mean()))

CV accuracy [ 0.76653696  0.7734375   0.78039216], Average accuracy 0.773455540614
Estimators 1, CV accuracy [ 0.6692607  0.6796875  0.6745098], Average accuracy 0.674486001437
Estimators 11, CV accuracy [ 0.73540856  0.74609375  0.78431373], Average accuracy 0.755272011934
Estimators 21, CV accuracy [ 0.75875486  0.73046875  0.77647059], Average accuracy 0.755231400683
Estimators 31, CV accuracy [ 0.72762646  0.7421875   0.77254902], Average accuracy 0.747454326251
Estimators 41, CV accuracy [ 0.73540856  0.74609375  0.78823529], Average accuracy 0.756579201476
Estimators 51, CV accuracy [ 0.72762646  0.75390625  0.78039216], Average accuracy 0.753974955336
Estimators 61, CV accuracy [ 0.73151751  0.7421875   0.78431373], Average accuracy 0.752672911739
Estimators 71, CV accuracy [ 0.74708171  0.765625    0.78823529], Average accuracy 0.766980668727
Estimators 81, CV accuracy [ 0.73151751  0.7421875   0.78823529], Average accuracy 0.753980101282
Estimators 91, CV accuracy [ 0.75875486

In [60]:
model = RandomForestClassifier(n_estimators = 100)
model.fit(X,y)
features = X.columns
feature_importances = model.feature_importances_
features_df = pd.DataFrame({'Features': features, 'Importance Score': feature_importances})
features_df.sort_values('Importance Score', inplace=True, ascending=False)

features_df.head()



,Features,Importance Score
1,plasma_glucose,0.303477
3,bmi,0.197455
4,diabetes_pedigree,0.148711
5,age,0.146220
2,diastolic_pressure,0.107164


In [64]:
model = RandomForestClassifier(n_estimators = 100) 
model.fit(X_train,y_train)
y_pred = model.predict(X_test) 
print metrics.classification_report(y_test, y_pred, target_names=['No Diabetes', 'Diabetes'])

#Precision of 0.75 and f1 of 0.74, so third best behind LR and kNN respectively.

             precision    recall  f1-score   support

No Diabetes       0.74      0.89      0.81       114
   Diabetes       0.78      0.54      0.64        78

avg / total       0.75      0.75      0.74       192



In [71]:
#- If you could only choose one, which classifer from the above that you already ran is best? How do you define
#best? (hint: could be prediction accuracy, running time, interpretability, etc)

#I would go with kNN based on best prediction accuracy (precision and f1 score).  It's also one of the fastest models.
#This data set is also very similar to the kNN flower example for classification.

In [72]:
%%timeit
knn = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train) 
y_pred = pima_knn.predict(X_test) 
y_train_pred = pima_knn.predict(X_train)

100 loops, best of 3: 4.17 ms per loop
